In [23]:
import numpy as np
import pandas as pd

In [7]:
def npv(cash_flows, rate):
    return sum(cf / (1 + rate) ** i for i, cf in enumerate(cash_flows))

In [92]:
def d_npv_dr(cash_flows, rate):
    try:
        return sum(-i * cf / (1 + rate) ** (i + 1) for i, cf in enumerate(cash_flows))
    except OverflowError:
        return float('inf')


In [93]:
# Function to calculate the second derivative of NPV
def d2_npv_dr2(cash_flows, rate):
    try:
        return sum(((i+1)*i*cf)/(1+rate) ** (i + 2) for i, cf in enumerate(cash_flows))
    except OverflowError:
        return float('inf')

In [94]:
def nr_irr(cash_flows, steps =0, initial_guess=-0.1, tolerance=1e-7, max_iterations=50):
    rate = initial_guess
    results = []
    if not any(cf > 0 for cf in cash_flows) or not any(cf < 0 for cf in cash_flows):
        raise ValueError("Cash flows must contain both positive and negative values.")
        
    for iteration in range(max_iterations):
        npv_value = npv(cash_flows, rate)
        d_npv_dr_value = d_npv_dr(cash_flows, rate)
        
        if abs(npv_value) < tolerance:
            if steps == 1:
                df_results = pd.DataFrame(results)
                return df_results
            else:
                return rate
        
        if d_npv_dr_value == float('inf') or d_npv_dr_value == 0:
            raise ValueError("Derivative is zero or overflow occurred. No solution found.")
        if rate > 10 or rate < -1:
            raise ValueError("Rate exceeded bounds. No solution found.")

        deltaIRR = npv_value / d_npv_dr_value
        
        rate -= deltaIRR

        results.append({
            'Iteration': iteration,
            'Rate': rate,
            'NPV': npv_value,
        })
    
    raise ValueError("Maximum iterations reached. No solution found.")

In [95]:
def halley_irr(cash_flows, steps = 0,initial_guess=-0.1, tolerance=1e-7, max_iterations=50):
    rate = initial_guess
    
    if not any(cf > 0 for cf in cash_flows) or not any(cf < 0 for cf in cash_flows):
        raise ValueError("Cash flows must contain both positive and negative values.")
        
    results = []
    
    for iteration in range(max_iterations):
        npv_value = npv(cash_flows, rate)
        d_npv_dr_value = d_npv_dr(cash_flows, rate)
        d2_npv_dr2_value = d2_npv_dr2(cash_flows, rate)
        
        if abs(npv_value) < tolerance:
            if steps == 1:
                df_results = pd.DataFrame(results)
                return df_results
            else:
                return rate
        
        if d_npv_dr_value == float('inf') or d_npv_dr_value == 0:
            raise ValueError("Derivative is zero or overflow occurred. No solution found.")
            
        if rate > 10 or rate < -1:
            raise ValueError("Rate exceeded bounds. No solution found.")

        deltaIRR = (2*npv_value*d_npv_dr_value) / ((2*(d_npv_dr_value ** 2) - (npv_value * d2_npv_dr2_value)))
        
        rate -= deltaIRR
        
        results.append({
            'Iteration': iteration,
            'Rate': rate,
            'NPV': npv_value,
        })
    
    raise ValueError("Maximum iterations reached. No solution found.")
        

In [96]:
def binr_irr(cash_flows, steps = 0, tolerance=1e-7, bisection_iteration=6, max_iterations=50):
    a=-0.9
    b=1.0

    tb_steps = []
    error = ''

    if not any(cf > 0 for cf in cash_flows) or not any(cf < 0 for cf in cash_flows):
        error ="Cash flows must contain both positive and negative values."
        if steps == 1:
            tb_steps.append({
            'Iteration': iteration,
            'Rate': rate,
            'NPV': npv_value,
            'Error': error,
            })
            df_results = pd.DataFrame(tb_steps)
            return df_results
        else:
            # return rate
            raise ValueError("Cash flows must contain both positive and negative values.")

    for iteration in range(bisection_iteration):
        c = (a+b)/2
        npva = npv(cash_flows, a)
        npvb = npv(cash_flows, b)
        npvc = npv(cash_flows, c)
        tb_steps.append({
            'Iteration': iteration,
            'a': a,
            'b': b,
            'c': c,
        })
        
        if npva * npvc < 0:
            b = c
        else:
            a = c

    rate = c
        
    for iteration in range(max_iterations):
        npv_value = npv(cash_flows, rate)
        d_npv_dr_value = d_npv_dr(cash_flows, rate)
        
        if abs(npv_value) < tolerance:
            if steps == 1:
                df_results = pd.DataFrame(tb_steps)
                return df_results
            else:
                return rate
        
        if d_npv_dr_value == float('inf') or d_npv_dr_value == 0:
            error = "Derivative is zero or overflow occurred. No solution found."
            if steps == 1:
                tb_steps.append({
                'Iteration': iteration,
                'Rate': rate,
                'NPV': npv_value,
                'Error': error,
                })
                df_results = pd.DataFrame(tb_steps)
                return df_results
            else:
                # return rate
                raise ValueError("Derivative is zero or overflow occurred. No solution found.")
                
        if rate > 10 or rate < -1:
            error = "Rate exceeded bounds. No solution found."
            
            if steps == 1:
                tb_steps.append({
                'Iteration': iteration,
                'Rate': rate,
                'NPV': npv_value,
                'Error': error,
                })
                df_results = pd.DataFrame(tb_steps)
                return df_results
            else:
                # return rate
                raise ValueError("Rate exceeded bounds. No solution found.")

        deltaIRR = npv_value / d_npv_dr_value
        
        rate -= deltaIRR

        tb_steps.append({
            'Iteration': iteration,
            'Rate': rate,
            'NPV': npv_value,
            'Error': error,
        })
    
    raise ValueError("Maximum iterations reached. No solution found.")
    
    
            

In [97]:
def process_cf_csv(csv_file):
    df = pd.read_csv(csv_file, header=None, names=['project_id', 'period', 'cashflow'])
    
    # Group by project_id
    grouped = df.groupby('project_id')
    
    # Initialize a list to store the results
    cf_table = []

    # Iterate through each group
    for project_id, group in grouped:
        # Sort by period just in case
        group = group.sort_values(by='period')
        
        # Extract the cashflows
        cash_flows = group['cashflow'].tolist()

        cf_table.append({
            'project_id': project_id,
            'cashflow': cash_flows,
        })
    df_cf = pd.DataFrame(cf_table)
    
    return df_cf
        


In [98]:
def calculate_project_irr(cf_df):
    # Initialize a list to store the results
    irr_results = []

    # Iterate through each row in the processed DataFrame
    for _, row in cf_df.iterrows():
        project_id = row['project_id']
        cash_flows = row['cashflow']
        
        # Calculate the IRR
        try:
            final_irr = halley_irr(cash_flows)
            irr_results.append({'project_id': project_id, 'IRR': final_irr})
        except ValueError as e:
            irr_results.append({'project_id': project_id, 'IRR': None, 'Error': str(e)})
    
    # Convert results to DataFrame
    irr_df = pd.DataFrame(irr_results)
    return irr_df

In [38]:
# Example usage
csv_file = '/Users/ryanabsar/Library/CloudStorage/OneDrive-BuroHappold/Documents/1_Project/3_DGCL/Documents/cashflowtest.csv'
cf_df = process_cf_csv(csv_file)
irr_results_df = calculate_project_irr(cf_df)
print(irr_results_df)


                 project_id       IRR  \
0               AddRev-DI-A  8.980442   
1              AddRev-DII-A  6.338857   
2    Cancelled-Assets-TDC-A       NaN   
3           Corporate-Items -0.005657   
4           D2-NewHotel-1-A       NaN   
..                      ...       ...   
762              WS-New-6-A       NaN   
763              WS-New-7-A       NaN   
764              WS-New-8-A -0.023705   
765              WS-New-9-A -0.079635   
766         WS-TOTAL-Land-A  0.342010   

                                                 Error  
0                                                  NaN  
1                                                  NaN  
2             Rate exceeded bounds. No solution found.  
3                                                  NaN  
4    Cash flows must contain both positive and nega...  
..                                                 ...  
762           Rate exceeded bounds. No solution found.  
763           Rate exceeded bounds. No solution fou

In [99]:
project_id = 'AddRev-DI-A'
cf_project = cf_df.loc[cf_df['project_id'] == project_id, 'cashflow'].values[0]
# irr = nr_irr(cf_df.iloc[7]['cashflow'],1)
irrBiNR = binr_irr(cf_project,1)
irrBiNR

,Iteration,a,b,c,Rate,NPV,Error
0,0,-0.900000,1.0,0.050000,NaN,NaN,NaN
1,1,0.050000,1.0,0.525000,NaN,NaN,NaN
2,2,0.525000,1.0,0.762500,NaN,NaN,NaN
3,3,0.762500,1.0,0.881250,NaN,NaN,NaN
4,4,0.881250,1.0,0.940625,NaN,NaN,NaN
5,5,0.940625,1.0,0.970313,NaN,NaN,NaN
6,0,NaN,NaN,NaN,1.278926,2.307715e+01,
7,1,NaN,NaN,NaN,1.650667,9.303146e+00,
8,2,NaN,NaN,NaN,2.092560,3.725562e+00,
9,3,NaN,NaN,NaN,2.612509,1.484619e+00,
